<a href="https://colab.research.google.com/github/bam396/nyu-python-public-policy/blob/master/HW4_old.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HOMEWORK 4 Coding: Analyzing and visualizing specific categories of 311 complaints

Create a new Google Colab notebook called "HW4".

**Problem Statement:** When comparing total 311 request counts per community district against the respective population, there seems to be a general trend that the higher the population, the higher the number of 311 requests. However, there is a lot of variance in the data. Maybe narrowing in on specific complaint type categories will help us better understand trends in 311 requests across the city.

"Noise" and "heat and hot water" were the most frequest requests in the entire 311 dataset. Let's try isolating the count of requests in each of those two categories versus all the other complaints.

**Step 1.** First you will need to prep your data to get the count of complaints in each of the three categories. 

In [0]:
import pandas as pd
from google.colab import drive

In [0]:
import matplotlib.pyplot as plt 
import numpy as np

In [0]:
# You can use pd.set_option() to make sure you see all the rows and columns in your dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
drive.mount('/content/drive')
# follow the link it generates, choose your account, and then paste in the authorization code it provides

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/My Drive/Data for Python/cleaned_311_data.csv', header='infer')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,18,21,32,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


You can update the code from lecture that gets the total count of 311 requests per CD to instead get the count of 311 requests per CD that meet the criteria for the 3 categories. You should create three seprate dataframes, one per category:
1.   complaint_type = 'Noise - Residential'
2.   complaint_type = 'HEAT/HOT WATER'
3.   Everything else: (complaint_type != 'Noise - Residential' and complaint_type != 'HEAT/HOT WATER')

In [0]:
df['complaint_type'] = df['complaint_type'].str.lower()

In [7]:
cb_counts = df.groupby('community_board').size().reset_index(name='count_of_311_requests')
print(cb_counts)

              community_board  count_of_311_requests
0               0 Unspecified                  39385
1                    01 BRONX                  27353
2                 01 BROOKLYN                  68101
3                01 MANHATTAN                  21259
4                   01 QUEENS                  60425
5            01 STATEN ISLAND                  65145
6                    02 BRONX                  20039
7                 02 BROOKLYN                  44061
8                02 MANHATTAN                  36688
9                   02 QUEENS                  35238
10           02 STATEN ISLAND                  40179
11                   03 BRONX                  29738
12                03 BROOKLYN                  66360
13               03 MANHATTAN                  51399
14                  03 QUEENS                  39240
15           03 STATEN ISLAND                  48729
16                   04 BRONX                  61086
17                04 BROOKLYN                 

In [27]:
def recode_borocd_counts(row):
    if row.community_board[0] != 'U':
      if 'MANHATTAN' in row.community_board:
        return '1' + row.community_board[:2]
      elif 'BRONX' in row.community_board:
        return '2' + row.community_board[:2]
      elif 'BROOKLYN' in row.community_board:
        return '3' + row.community_board[:2]
      elif 'QUEENS' in row.community_board:
        return '4' + row.community_board[:2]
      elif 'STATEN ISLAND' in row.community_board:
        return '5' + row.community_board[:2]
    else:
      return 'Invalid borocd'

df['borocd'] = df.apply(recode_borocd_counts, axis=1)

# checking recode
df.head()

,Unnamed: 0,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_(state_plane),y_coordinate_(state_plane),open_data_channel_type,park_facility_name,park_borough,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,borocd
0,0,39888071,08/01/2018 12:00:10 AM,08/01/2018 01:52:46 AM,DHS,Operations Unit - Department of Homeless Services,homeless person assistance,NaN,Other,10029,200 EAST 109 STREET,EAST 109 STREET,3 AVENUE,2 AVENUE,NaN,NaN,ADDRESS,NEW YORK,NaN,NaN,Closed,08/01/2018 01:00:10 AM,The mobile outreach response team offered serv...,08/01/2018 01:52:46 AM,11 MANHATTAN,1.016560e+09,MANHATTAN,1000049.0,228317.0,MOBILE,Unspecified,MANHATTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.793339,-73.942942,"(40.79333937834769, -73.9429417746998)",111
1,1,39889166,08/01/2018 12:00:26 AM,08/18/2018 10:46:43 AM,HPD,Department of Housing Preservation and Develop...,door/window,DOOR,RESIDENTIAL BUILDING,10031,528 WEST 136 STREET,WEST 136 STREET,NaN,NaN,NaN,NaN,ADDRESS,NEW YORK,NaN,NaN,Closed,NaN,The Department of Housing Preservation and Dev...,08/18/2018 10:46:43 AM,09 MANHATTAN,1.019880e+09,MANHATTAN,997239.0,238074.0,PHONE,Unspecified,MANHATTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.820124,-73.953071,"(40.82012422332215, -73.9530712339799)",109
2,2,39882869,08/01/2018 12:00:54 AM,08/01/2018 12:49:55 AM,NYPD,New York City Police Department,noise - residential,Loud Music/Party,Residential Building/House,11216,761 LINCOLN PLACE,LINCOLN PLACE,ROGERS AVENUE,NOSTRAND AVENUE,NaN,NaN,ADDRESS,BROOKLYN,NaN,Precinct,Closed,08/01/2018 08:00:54 AM,The Police Department responded to the complai...,08/01/2018 12:49:55 AM,08 BROOKLYN,3.012540e+09,BROOKLYN,997732.0,183674.0,MOBILE,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.670809,-73.951399,"(40.67080917938279, -73.9513990916184)",308
3,3,39894246,08/01/2018 12:01:00 AM,08/02/2018 10:30:00 PM,DEP,Department of Environmental Protection,noise,Noise: Construction Before/After Hours (NM1),NaN,10010,NaN,NaN,NaN,NaN,5 AVENUE,EAST 21 STREET,INTERSECTION,NEW YORK,NaN,NaN,Closed,NaN,The Department of Environmental Protection did...,08/02/2018 10:30:00 PM,05 MANHATTAN,NaN,MANHATTAN,986878.0,208974.0,PHONE,Unspecified,MANHATTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.740262,-73.990517,"(40.74026158873342, -73.99051651686905)",105
4,4,39881329,08/01/2018 12:01:00 AM,08/05/2018 12:00:00 AM,DSNY,Department of Sanitation,request large bulky item collection,Request Large Bulky Item Collection,Sidewalk,11413,121-28 198 STREET,198 STREET,120 AVENUE,122 AVENUE,NaN,NaN,ADDRESS,Springfield Gardens,NaN,NaN,Closed,NaN,NaN,08/05/2018 12:00:00 AM,12 QUEENS,4.126830e+09,QUEENS,1053308.0,190084.0,PHONE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.688144,-73.750990,"(40.68814402968042, -73.75098958473612)",412


In [0]:

# this function will recode the community_board value into borocd in a new column
  def recode_borocd_counts(row):
    if row.community_board[0] != 'U':
      if 'MANHATTAN' in row.community_board:
        return '1' + row.community_board[:2]
      elif 'BRONX' in row.community_board:
        return '2' + row.community_board[:2]
      elif 'BROOKLYN' in row.community_board:
        return '3' + row.community_board[:2]
      elif 'QUEENS' in row.community_board:
        return '4' + row.community_board[:2]
      elif 'STATEN ISLAND' in row.community_board:
        return '5' + row.community_board[:2]
    else:
      return 'Invalid borocd'

cb_counts['borocd'] = cb_counts.apply(recode_borocd_counts, axis=1)

In [0]:
merged_data = pd.merge(left=cb_counts, right=df, left_on=['borocd','community_board'], right_on=['borocd','community_board'])

In [42]:
merged_data_noise = merged_data[merged_data.complaint_type=='noise - residential']
merged_data_noise = merged_data_noise.groupby('borocd').size().reset_index(name='count_of_311_requests')
#checking code
print(len(merged_data_noise))

68


In [43]:
print(merged_data_noise)

   borocd  count_of_311_requests
0     101                    722
1     102                   1837
2     103                   6272
3     104                   2702
4     105                    741
5     106                   2078
6     107                   4297
7     108                   2283
8     109                   4586
9     110                   7659
10    111                   4953
11    112                  11755
12    164                      3
13    201                   3691
14    202                   1939
15    203                   4238
16    204                   7407
17    205                   6819
18    206                   3871
19    207                   9207
20    208                   4406
21    209                   5037
22    210                   2305
23    211                   5227
24    212                   5585
25    226                      1
26    227                      1
27    228                      2
28    301                   5844
29    302 

In [44]:
merged_data_heat = merged_data[merged_data.complaint_type=='heat/hot water']
merged_data_heat = merged_data_heat.groupby('borocd').size().reset_index(name='count_of_311_requests')
#checking code
print(len(merged_data_heat))

62


In [45]:
merged_data_else = merged_data[merged_data.complaint_type != 'noise - residential'][merged_data.complaint_type != 'heat/hot water']
merged_data_else = merged_data_else.groupby('borocd').size().reset_index(name='count_of_311_requests')
#checking code
print(len(merged_data_else))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


72


In [46]:
# per the starter template, checking all
merged_data_all = merged_data.groupby('borocd').size().reset_index(name='count_of_311_requests')
#checking code
print(len(merged_data_all))

72


In [0]:
valid_borocds = ['101','102','103','104','105','106','107','108','109','110','111','112','201','202','203','204','205','206','207','208','209','210','211','212','301','302','303','304','305','306','307','308','309','310','311','312','313','314','315','316','317','318','401','402','403','404','405','406','407','408','409','410','411','412','413','414','501','502','503']

In [48]:
merged_data_noise = merged_data_noise[merged_data_noise.borocd.isin(valid_borocds)]
print(len(merged_data_noise))

merged_data_heat = merged_data_heat[merged_data_heat.borocd.isin(valid_borocds)]
print(len(merged_data_heat))

merged_data_else = merged_data_else[merged_data_else.borocd.isin(valid_borocds)]
print(len(merged_data_else))

merged_data_all = merged_data_all[merged_data_all.borocd.isin(valid_borocds)]
print(len(merged_data_all))

59
59
59
59


**Step 2.** Create a stacked bar chart comprised of three stacked bars. The total height of each bar (the sum of the three stacked bars) should reflect the total count of 311 complaints per CD.

1. number of 'Noise - Residential' complaints per CD
2. number of 'HEAT/HOT WATER' complaints per CD
3. number of other complaints per CD